# Bioinformatics Project 2025 - Motif CNN & GO Prediction

**Course:** GRS34806 Deep Learning

**Authors:** ............

**Date:**



## Importing Libraries

In [3]:
import os
import sys
from pathlib import Path

! git clone https://git.wur.nl/bioinformatics/grs34806-deep-learning-project-data.git -q
! git clone https://github.com/maussn/GRS34806-project.git -q
os.chdir(Path('grs34806-deep-learning-project-data'))

fatal: destination path 'grs34806-deep-learning-project-data' already exists and is not an empty directory.
fatal: destination path 'GRS34806-project' already exists and is not an empty directory.


In [2]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter
from sklearn.metrics import precision_recall_fscore_support
from datetime import datetime
import sys
import os
from pathlib import Path
import random
import seaborn as sns

## Data I/O & Tokenisation

In [4]:
# 1. read() --------------------------------------------------------------------
def read(seqfile: str, posfile: str) -> tuple[list, list]:
    """Read file with sequences and file with positive cases into lists.

    :param seqfile: file with sequences
    :type seqfile: str
    :param posfile: file with positive cases (annotated with function)
    :type posfile: str
    :return: two lists, first with sequences and second with boolean labels
    :rtype: tuple[list, list]
    """
    idlist = []
    datalist = []
    labellist = []
    with open(seqfile, 'r') as f:
        for line in f.readlines():
            line = line.rstrip().split('\t')
            idlist.append(line[0])
            datalist.append(line[1])
            labellist.append(False)
    with open(posfile, 'r') as f:
        for line in f.readlines():
            id = line.rstrip()
            try:
                i = idlist.index(id)
                labellist[i] = True
            except ValueError:
                continue
    return datalist, labellist


# 2. split_labelled() ----------------------------------------------------------
def split_labelled(datalist: list, labellist: list):
    """Return two separate sequence lists: positives & negatives."""
    pos_datalist = []
    neg_datalist = []
    for i, label in enumerate(labellist):
        if label:
            pos_datalist.append(datalist[i])
        else:
            neg_datalist.append(datalist[i])
    return pos_datalist, neg_datalist


# 3. remove_sequences() -----
def remove_sequences(datalist: list, fraction=0.5):
    """Randomly keeps half of the list"""
    random.shuffle(datalist)
    keep = round(len(datalist) * fraction)
    return datalist[:keep]


# 4. fuse_sequence_lists() ------------
def fuse_sequence_lists(pos_datalist: list, neg_datalist: list):
    """Merge postives and negetaves into one list + label"""
    pos_labels = [True for _ in pos_datalist]
    neg_labels = [False for _ in neg_datalist]
    datalist = pos_datalist + neg_datalist
    labellist = pos_labels + neg_labels
    return datalist, labellist


# 5. generate_train_test() --------
def generate_train_test(datalist: list, labellist: list, fraction: float=0.8):
    """Split up dataset in training set and test set

    :param datalist: list with sequences
    :type datalist: list
    :param labellist: list with labels
    :type labellist: list
    :param ratio: fraction to be added to the training set, remainder is added to the test set, defaults to 0.8
    :type ratio: float, optional
    :return: four lists, first two the training data and labels, second two the test data and labels
    :rtype: tuple[list, list, list, list]
    """
    c = list(zip(datalist, labellist))
    random.shuffle(c)
    datalist[:], labellist[:] = zip(*c)
    i = round(len(datalist) * fraction)
    traindatalist = datalist[:i]
    trainlabellist = labellist[:i]
    testdatalist = datalist[i:]
    testlabellist = labellist[i:]
    return traindatalist, trainlabellist,testdatalist,testlabellist


# 6. Tokenisation & Padding --------
def tokenize(data: list, map2num: dict, non_aa_num: int=20) -> list:
    """Tokenize all sequences in a list

    :param data: list of sequences to tokenize
    :type data: list
    :param map2num: ammino acid -> integer token mapping
    :type map2num: dict
    :param non_aa_num: token for non amino acid characters, defaults to 20
    :type non_aa_num: int, optional
    :return: list of tokenized sequences
    :rtype: list
    """
    seq = []
    for count, i in enumerate(data):
        seq.append([map2num.get(j,non_aa_num) for j in list(i)])
    return seq


def truncate_pad(line: list, num_steps: int, padding_token: int) -> list:
    """Truncate or pad a tokenized sequence

    :param line: tokenized sequence
    :type line: list
    :param num_steps: maximum sequence length
    :type num_steps: int
    :param padding_token: token to be used for padding
    :type padding_token: int
    :return: truncated/padded sequence
    :rtype: list
    """
    if len(line) > num_steps:
        return line[:num_steps] # Truncate
    return line + [padding_token] * (num_steps - len(line)) # Pad


def build_seq_array(lines: list, num_steps: int, non_aa_num: int=20) -> torch.tensor:
    """Truncate or pad tokenized sequences and convert to tensor

    :param lines: tokenized sequences
    :type lines: list
    :param num_steps: maximum sequence length
    :type num_steps: int
    :param non_aa_num: token for padding, defaults to 20
    :type non_aa_num: int, optional
    :return: tensor with truncated/padded tokenized sequences
    :rtype: torch.tensor
    """
    return torch.tensor([truncate_pad(l, num_steps, non_aa_num) for l in lines], dtype=torch.long)


# 7. load_array() & load_data()
def load_array(data_arrays: tuple[torch.tensor, torch.tensor], batch_size: int, is_train: bool=True) -> torch.utils.data.DataLoader:
    """Construct a PyTorch data iterator.

    Taken from d2l package"""
    dataset = torch.utils.data.TensorDataset(*data_arrays)
    return torch.utils.data.DataLoader(dataset, batch_size, shuffle=is_train)


def load_data(batch_size: int, num_steps: int, dataset: tuple[list, list]) -> torch.utils.data.DataLoader:
    """Tokenize sequence/label dataset and load into dataloader.

    :param batch_size: size of each batch
    :type batch_size: int
    :param num_steps: maximum sequence length
    :type num_steps: int
    :param dataset: first list contains sequences, second labels
    :type dataset: tuple[list, list]
    :return: torch dataloader which gives a tensor of sequences in a batch and a tensor with their labels
    :rtype: torch.utils.data.DataLoader
    """
    mapaa2num = {aa: i for (i, aa) in enumerate(list("ACDEFGHIKLMNPQRSTVWY"))}
    seq,lab = dataset
    seq = tokenize(seq, mapaa2num)
    seq_array = build_seq_array(seq, num_steps)
    data_arrays = (seq_array, torch.tensor(lab, dtype=torch.long))
    data_iter = load_array(data_arrays, batch_size)
    return data_iter

## Dataset selector





In [5]:
dataset_id = "len100_200_n1000"  # select any of the 6's IDs
REMOVE = None                    # None | "neg" | "pos"

seq_path  = f"{dataset_id}.seq"
pos_path  = f"{dataset_id}.pos"
datalist, labellist = read(seq_path, pos_path)

# Removing half operation
if REMOVE is not None:
    pos_datalist, neg_datalist = split_labelled(datalist, labellist)
    if REMOVE == "neg":
        neg_datalist = remove_sequences(neg_datalist, 0.5)
    elif REMOVE == "pos":
        pos_datalist = remove_sequences(pos_datalist, 0.5)
    datalist, labellist = fuse_sequence_lists(pos_datalist, neg_datalist)

## Data Loader

In [6]:
batch_size = 10
num_steps = 500 # max sequence length

traindatalist, trainlabellist, testdatalist, testlabellist = generate_train_test(datalist, labellist, 0.8)
traindataset = [traindatalist, trainlabellist]
testdataset = [testdatalist, testlabellist]

# Set batch_size and num_steps (maximum sequence length)
train_iter = load_data(batch_size, num_steps, traindataset)
test_iter = load_data(batch_size, num_steps, testdataset)

print("batch shape  :", next(iter(train_iter))[0].shape)
print("labels shape :", next(iter(train_iter))[1].shape)

batch shape  : torch.Size([10, 500])
labels shape : torch.Size([10])


## Training / Evaluation

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [19]:
def init_weights(layer):
    if type(layer) == nn.Linear or type(layer) == nn.Conv1d:
        nn.init.xavier_uniform_(layer.weight)

class Trainer:
    def __init__(self, model, loss_fn, optimizer, device='cuda'):
        self.model = model.to(device)
        self.loss_fn = loss_fn
        self.optimizer = optimizer
        self.device = device

    # one epoch
    def _train_one_epoch(self, train_iter):
        """
        Run one epoch and return mean epoch loss.
        """
        self.model.train()
        epoch_loss= 0.0
        n_steps = 0

        for inputs, labels in train_iter:
            inputs = inputs.to(self.device)
            labels = labels.to(self.device)


            self.optimizer.zero_grad()
            outputs = self.model(inputs)
            loss = self.loss_fn(outputs, labels)
            loss.backward()
            self.optimizer.step()

            epoch_loss += loss.item()
            n_steps += 1

        return epoch_loss / n_steps   # mean epoch loss


    # full training

    def train(self, epochs, train_iter, test_iter):
        """
        Finds the best accuracy observed.
        """
        best_acc = 0.0

        for epoch in range(1, epochs + 1):
            train_loss = self._train_one_epoch(train_iter)

            test_loss, test_acc, test_prec, test_rec, test_f1 = \
                    self.evaluate(test_iter, calc_loss=True)

            print(f"[epoch {epoch:02d}] "
                  f"train‑loss={train_loss:.4f} | "
                  f"test‑loss={test_loss:.4f} | "
                  f"test‑acc={test_acc:.4f} | "
                  f"P={test_prec:.4f} | R={test_rec:.4f} | F1={test_f1:.4f}")

            if test_acc > best_acc:
                best_acc = test_acc

        return best_acc


##### dit is gemaakt door AI!########### CHANGE
    def evaluate(self, data_iter, calc_loss=False):
        """
        Returns (loss, acc, prec, rec, f1) **if** calc_loss is True
        otherwise returns (0.0, acc, prec, rec, f1)
        so the caller can always unpack 5 values.
        """
        self.model.eval()
        all_pred, all_gold = [], []
        loss_total, n_batches = 0.0, 0

        with torch.no_grad():
            for x, y in data_iter:
                x, y          = x.to(self.device), y.to(self.device)
                logits        = self.model(x)
                pred          = logits.argmax(dim=1)

                all_pred.append(pred.cpu())
                all_gold.append(y.cpu())

                if calc_loss:
                    loss_total += self.loss_fn(logits, y).item()
                    n_batches  += 1

        all_pred = torch.cat(all_pred)
        all_gold = torch.cat(all_gold)

        acc  = (all_pred == all_gold).float().mean().item()
        prec, rec, f1, _ = precision_recall_fscore_support(
            all_gold, all_pred, average='micro'
        )

        avg_loss = loss_total / n_batches if (calc_loss and n_batches) else 0.0
        return avg_loss, acc, prec, rec, f1

In [20]:
class BerryCNN1D(nn.Module):
    """1D Convolutional Neural Network for protein function classification"""
    def __init__(self, context_size: int, vocab_size: int = 21,
                 dropout_rate = 0, conv_channels: int = 128,
                 use_bias: bool = False):

        super().__init__()
        assert context_size % 2 == 0, f'Invalid context_size, {context_size} is not an even number'

        self.embedding = nn.Embedding(vocab_size, vocab_size)

        # CNN model for binary classification
        self.conv1 = nn.Sequential(
            # conv block 1
            nn.Conv1d(in_channels=vocab_size, out_channels=conv_channels, kernel_size=3, padding='same', bias=use_bias),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2)
        )
        self.conv2 = nn.Sequential(
            # conv block 2
            nn.Conv1d(in_channels=conv_channels, out_channels=conv_channels, kernel_size=3, padding='same', bias=use_bias),
            nn.ReLU(),
            nn.AdaptiveMaxPool1d(1)
        )


        self.dropout = nn.Dropout(dropout_rate)
        self.fc = nn.Sequential(
            # flatten + classification head
            nn.Flatten(1, -1),
            nn.LazyLinear(out_features=2, bias=use_bias)  # binary classification
        )


    def forward(self, x: torch.tensor, targets: torch.tensor = None):
        """Predict protein function class (0 or 1)"""
        x = self.embedding(x).permute(0,2,1)
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.dropout(x)
        return self.fc(x)


In [21]:
# Selecting the CNN
net = BerryCNN1D(context_size=num_steps, vocab_size=21, conv_channels=128)

## Training the Model

In [24]:
from sklearn.model_selection import ParameterGrid

param_grid = {
    'dropout_rate': [0, 0.3, 0.6],
    'lr': [0.01, 0.001],
    'momentum': [0, 0.5, 0.9]
}

grid = list(ParameterGrid(param_grid))

best_acc = 0
best_params = None

for params in grid:
    model = BerryCNN1D(
        context_size=num_steps,
        vocab_size=21,
        dropout_rate=params['dropout_rate'],
        conv_channels=128
    )
    model.apply(init_weights)

    optimizer = torch.optim.SGD(
        model.parameters(),
        lr=params['lr'],
        momentum=params['momentum']
    )

    loss_fn = nn.CrossEntropyLoss()
    trainer = Trainer(model, loss_fn, optimizer)

    acc = trainer.train(epochs=5, train_iter=train_iter,
                    test_iter=test_iter)

    if acc > best_acc:
        best_acc    = acc
        best_params = params
        print(f"New best accuracy {best_acc:.4f} with {best_params}")

print("Best hyper‑parameters found:", best_params)

[epoch 01] train‑loss=0.5849 | test‑loss=0.3866 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 02] train‑loss=0.1818 | test‑loss=0.0800 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 03] train‑loss=0.0550 | test‑loss=0.0346 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 04] train‑loss=0.0284 | test‑loss=0.0213 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 05] train‑loss=0.0183 | test‑loss=0.0137 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
New best accuracy 1.0000 with {'dropout_rate': 0, 'lr': 0.01, 'momentum': 0}
[epoch 01] train‑loss=0.5275 | test‑loss=0.1575 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 02] train‑loss=0.0638 | test‑loss=0.0278 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 03] train‑loss=0.0176 | test‑loss=0.0130 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 04] train‑loss=0.0093 | test‑loss=0.0082 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 05] train‑loss=0